In [ ]:
# =========================================================
#               설정(필요 시만 수정)
# =========================================================
from pathlib import Path
DATA_ROOT = Path(r"embbeding_data")   # embbeding_data 폴더
AGG_METHOD = "mean"           # "mean" | "max" | "flatten"
BATCH      = 128
EPOCHS     = 40
LR         = 0.005
DEVICE     = "cuda:0" if __import__("torch").cuda.is_available() else "cpu"
SAVE_DIR   = Path(r"D:\golf\fusion_ckpt")     # None → 저장 생략
# =========================================================

import json, time, numpy as np, torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score

# ---------- 데이터 로드 --------------------------------------------------
def load_split(split: str, model: str):
    """split=train|valid, model=stgcnpp|timesformer"""
    base = DATA_ROOT / model / split
    emb  = np.load(base / "embeddings.npy")
    lbl  = np.load(base / "labels.npy").ravel()
    return emb, lbl

ts_tr, y_tr = load_split("train", "timesformer")   # (N_tr, 768)
ts_va, y_va = load_split("valid", "timesformer")
st_tr, _    = load_split("train", "stgcnpp")       # (N_tr, 10, 256)
st_va, _    = load_split("valid", "stgcnpp")

assert (y_tr == np.load(DATA_ROOT/"stgcnpp/train/labels.npy").ravel()).all()
assert (y_va == np.load(DATA_ROOT/"stgcnpp/valid/labels.npy").ravel()).all()
print(f"✅ 로드 완료: train={ts_tr.shape[0]}  valid={ts_va.shape[0]}")


# ---------- ST-GCN 임베딩 (N,10,256) → (N,256 또는 2560) ---------------
def reduce_stgcn(arr):
    if AGG_METHOD == "mean":
        return np.nanmean(arr, axis=1)         # (N, 256)
    if AGG_METHOD == "max":
        return np.nanmax(arr, axis=1)
    if AGG_METHOD == "flatten":
        return arr.reshape(arr.shape[0], -1)   # (N, 2560)
    raise ValueError(f"AGG_METHOD={AGG_METHOD}")

st_tr_flat = reduce_stgcn(st_tr)
st_va_flat = reduce_stgcn(st_va)

# ---------- 특징 결합 ----------------------------------------------------
X_tr = np.concatenate([ts_tr, st_tr_flat], axis=1).astype(np.float32)
X_va = np.concatenate([ts_va, st_va_flat], axis=1).astype(np.float32)
y_tr = y_tr.astype(np.int64)
y_va = y_va.astype(np.int64)

print(f"Fused dims → train:{X_tr.shape}, valid:{X_va.shape}")


# ---------- PyTorch Dataset / DataLoader --------------------------------
ds_tr = TensorDataset(torch.from_numpy(X_tr), torch.from_numpy(y_tr))
ds_va = TensorDataset(torch.from_numpy(X_va), torch.from_numpy(y_va))
dl_tr = DataLoader(ds_tr, batch_size=BATCH, shuffle=True,  drop_last=False)
dl_va = DataLoader(ds_va, batch_size=BATCH, shuffle=False, drop_last=False)


# ---------- 모델 ---------------------------------------------------------
class FusionMLP(nn.Module):
    def __init__(self, in_dim, n_cls, hidden=(512, 128), p_drop=0.3):
        super().__init__()
        layers, dims = [], [in_dim, *hidden]
        for d_in, d_out in zip(dims[:-1], dims[1:]):
            layers += [nn.Linear(d_in, d_out), nn.ReLU(), nn.Dropout(p_drop)]
        layers += [nn.Linear(dims[-1], n_cls)]
        self.net = nn.Sequential(*layers)

    def forward(self, x):            # (B, in_dim)
        return self.net(x)

n_cls = int(max(y_tr.max(), y_va.max())) + 1
model = FusionMLP(in_dim=X_tr.shape[1], n_cls=n_cls).to(DEVICE)
opt    = torch.optim.AdamW(model.parameters(), lr=LR)
crit   = nn.CrossEntropyLoss()

print(f"🔧 모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")


# ---------- 학습 / 평가 루프 -------------------------------------------
best_acc, best_state = 0.0, None
for epoch in range(1, EPOCHS+1):
    # --- train ---
    model.train()
    loss_sum, n_correct, n_total = 0.0, 0, 0
    for x, y in dl_tr:
        x, y = x.to(DEVICE), y.to(DEVICE)
        opt.zero_grad()
        out  = model(x)
        loss = crit(out, y)
        loss.backward()
        opt.step()

        loss_sum  += loss.item() * y.size(0)
        n_correct += (out.argmax(1) == y).sum().item()
        n_total   += y.size(0)

    tr_loss = loss_sum / n_total
    tr_acc  = n_correct / n_total

    # --- valid ---
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x, y in dl_va:
            x = x.to(DEVICE)
            out = model(x)
            y_true.append(y)
            y_pred.append(out.argmax(1).cpu())
    y_true = torch.cat(y_true).numpy()
    y_pred = torch.cat(y_pred).numpy()
    va_acc = accuracy_score(y_true, y_pred)

    print(f"[{epoch:02d}/{EPOCHS}]  "
          f"loss={tr_loss:.4f}  train_acc={tr_acc*100:.2f}%  "
          f"valid_acc={va_acc*100:.2f}%")

    if va_acc > best_acc:
        best_acc, best_state = va_acc, model.state_dict()

# ---------- 최적 모델 저장 ----------------------------------------------
if SAVE_DIR:
    SAVE_DIR.mkdir(parents=True, exist_ok=True)
    torch.save(best_state, SAVE_DIR / "fusion_mlp_best.pth")
    with open(SAVE_DIR / "meta.json", "w", encoding="utf-8") as f:
        json.dump({
            "input_dim": X_tr.shape[1],
            "n_classes": n_cls,
            "valid_acc": best_acc,
            "agg_method": AGG_METHOD,
            "hidden": [512, 128],
            "epochs": EPOCHS
        }, f, indent=2)
    print(f"\n✅ 가중치 저장 → {SAVE_DIR/'fusion_mlp_best.pth'}")

# ---------- 최종 리포트 --------------------------------------------------
print("\n" + classification_report(y_true, y_pred, digits=4))
print(f"🏅 Best VALID ACC = {best_acc*100:.2f}%")


✅ 로드 완료: train=392  valid=44
Fused dims → train:(392, 1024), valid:(44, 1024)


C:\Users\qqppq\AppData\Local\Temp\ipykernel_22352\4148548505.py:40: RuntimeWarning: Mean of empty slice
  return np.nanmean(arr, axis=1)         # (N, 256)


🔧 모델 파라미터 수: 590,722
[01/40]  loss=nan  train_acc=47.96%  valid_acc=45.45%
[02/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[03/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[04/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[05/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[06/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[07/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[08/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[09/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[10/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[11/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[12/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[13/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[14/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[15/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[16/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[17/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[18/40]  loss=nan  train_acc=50.51%  valid_acc=45.45%
[19/40]

c:\Users\qqppq\anaconda3\envs\fusion\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\qqppq\anaconda3\envs\fusion\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\qqppq\anaconda3\envs\fusion\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.sh

In [13]:
import numpy as np, pandas as pd
root = Path(r"embbeding_data")
y_ts = np.load(root/'timesformer/train/labels.npy').ravel().astype(int)
y_st = np.load(root/'stgcnpp/train/labels.npy').ravel().astype(int)

mismatch_idx = np.where(y_ts != y_st)[0]
print(f"⚠️  서로 다른 위치: {mismatch_idx[:10]} ... 총 {len(mismatch_idx)}개")
pd.DataFrame({'idx': mismatch_idx,
              'ts_label': y_ts[mismatch_idx],
              'st_label': y_st[mismatch_idx]})


⚠️  서로 다른 위치: [] ... 총 0개


,idx,ts_label,st_label
